In [78]:
using DIVAnd
using PyPlot
using Proj4
using DelimitedFiles
using PyCall
using Dates
using NCDatasets
include("../scripts/PhytoInterp.jl")
doplot = true

true

## Packages for plotting

In [79]:
if doplot
    ccrs = pyimport("cartopy.crs")
    gridliner = pyimport("cartopy.mpl.gridliner")
    cfeature = pyimport("cartopy.feature")
    mticker = pyimport("matplotlib.ticker")
    myproj = ccrs.PlateCarree()
    coast = cfeature.GSHHSFeature(scale="full");
    mpl = pyimport("matplotlib");
    cartopyticker = pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
end 

PyObject <cartopy.mpl.ticker.LatitudeFormatter object at 0x7fe291d272d0>

## Files and directories

In [83]:
datadir = "/data/EMODnet/Biology/phytoplankton/csv/"
datafilelist = readdir(datadir);
@info("Working on $(length(datafilelist)) files");
figdir = "./figures/"
outputdir = "../product/"
#datafile = joinpath(datadir, "Biddulphia_sinensis1995-2020.csv")
speciesname = "Biddulphia_sinensis"
isdir(datadir) ? " " : mkpath(datadir)
isdir(figdir) ? " " : mkpath(figdir)
#isfile(datafile) ? @info("File already downloaded") : download("https://dox.ulg.ac.be/index.php/s/VgLglubaTLetHzc/download", datafile)

┌ Info: Working on 200 files
└ @ Main In[83]:3


" "

## Prepare mask
### Interpolation grid

In [84]:
longrid = -2.:0.1:10.
latgrid = 51.:0.1:56.

51.0:0.1:56.0

### Download file

In [85]:
bathname = joinpath(datadir, "gebco_30sec_4.nc")
if !isfile(bathname)
    download("https://dox.ulg.ac.be/index.php/s/RSwm4HPHImdZoQP/download", bathname)
else
    @info("Bathymetry file already downloaded")
end

"/data/EMODnet/Biology/phytoplankton/csv/gebco_30sec_4.nc"

### Read bathymetry

In [ ]:
bx, by, b = load_bath(bathname, true, longrid, latgrid)
@show size(b)

if doplot
    fig = PyPlot.figure()
    ax = PyPlot.subplot(111, projection=myproj)
    pcolor(bx,by,b', vmin=0.); 
    colorbar(orientation="horizontal")
    title("Depth")
    decorate_map(ax)
    PyPlot.savefig(joinpath(figdir, "northsea_bathymetry.png"), 
        dpi=300, bbox_inches="tight")
    #PyPlot.show()
    PyPlot.close()
end

### Metrics

In [86]:
mask, (pm, pn),(xi, yi) = DIVAnd.DIVAnd_rectdom(longrid, latgrid);
xi, yi, mask = DIVAnd.load_mask(bathname, true, longrid, latgrid, 0.0);
xx, yy = ndgrid(xi, yi);

## Main loop

In [95]:
include("../scripts/PhytoInterp.jl")

plot_heatmap (generic function with 2 methods)

In [ ]:
for datafile in datafilelist[1:20]
    speciesname = get_species_name(datafile)
    @debug("Working on $(speciesname)")
    
    # Data reading
    dates, lons, lats, occurs = read_data_phyto(joinpath(datadir, datafile));
    
    # Conversion of coordinates
    # We have to go from `EPSG:32361` to `WGS84`.         
    lon, lat = transform_coords(lons, lats)
    
    # Plot presence/absence
    if doplot
        make_plot_presence_absence(lon, lat, joinpath(figdir, "$(speciesname)_data_presence_absence.png"))
    end
    
    data_presence = occurs .== 1;
    data_absence = .!(data_presence);
    
    # Compute heatmap
    inflation = ones(length(lon));
    @time dens1, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon, lat), inflation, 0.2);
    @time dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon[data_presence], lat[data_presence]), inflation, 0.2);
    @time dens3, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm,pn), (xx, yy), 
        (lon[data_absence], lat[data_absence]), inflation, 0.2);
    
    if doplot
        """
        plot_heatmap(longrid, latgrid, dens1, lon[data_presence], lat[data_presence],
            "$(speciesname): heatmap with 'presence' data points", joinpath(figdir, "$(speciesname)_heatmap_presence.png"))
        plot_heatmap(longrid, latgrid, dens2, lon[data_absence], lat[data_absence],
            "$(speciesname): heatmap with 'absence' data points", joinpath(figdir, "$(speciesname)_heatmap_absence.png"))
        plot_heatmap(longrid, latgrid, dens3, lon, lat,
            "$(speciesname): heatmap with all data points", joinpath(figdir, "$(speciesname)_heatmap_all.png"))
        """
        plot_heatmap(longrid, latgrid, dens2 ./ (dens2.+dens3), lon[data_presence], lat[data_presence],
        "$(speciesname): normalised heatmap (presence/presence+absence)", joinpath(figdir, "$(speciesname)_heatmap_relative.png"))
    end
    
    # Write in netCDF
    create_nc_results(joinpath(outputdir, "$(speciesname)_heatmap.nc"), 
        longrid, latgrid, dens2 ./ (dens2 .+ dens3), speciesname);
 
end